In [ ]:
reset -f


In [ ]:
#!pip install pyarrow


In [ ]:
# http://www.sahko.tk/
# https://energiavirasto.fi/sahkon-hintatilastot
# https://energiavirasto.fi/documents/11120570/13026619/Tarjoushinnat/f3187958-9592-b3c9-217c-f35f6a41c60c?t=1701954573719


In [ ]:
import os
import re
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure


In [ ]:
input_folder: str = 'InputData/Spot_Price_Files'
preprocessed_folder: str = 'PreprocessedData'
input_file_list: list = list()
spot_price_file_name_gzip: str = 'Spot_Price_Table.gzip'
spot_price_file_name_csv: str = 'Spot_Price_Table.csv'
    

In [ ]:
subset_dates: dict = {'start_date': '2022-06-15', 'end_date': '2022-12-31'}


In [ ]:
def create_spot_price_dataframe() -> pd.DataFrame:
    input_file_list: list = os.listdir(input_folder)

    df_list: list = list()
    for input_file in input_file_list:
        m: re.Match|None = re.match('^.+\.csv$', input_file)
        if m is not None:
            file_path: str = os.path.join(input_folder, input_file)
            df: pd.DataFrame = pd.read_csv(file_path)
            df_list.append(df)
    df_spot_price = pd.concat(df_list)
    df_spot_price.drop_duplicates(inplace=True)
    return df_spot_price


In [ ]:
def convert_column_types(df: pd.DataFrame):
#     df['DateTime'] = pd.to_datetime(df['DateTime'], utc=True).dt.tz_convert('Europe/Helsinki')
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df.sort_values(['DateTime'], inplace=True, ascending=True)


In [ ]:
def calculate_alv_0_based_on_date_and_time(df: pd.DataFrame) -> None:
    start_date: str = '2022-12-01'
    end_date: str = '2023-04-30'
    df['alv'] = np.where((df['DateTime'].dt.strftime('%Y-%m-%d') >= start_date) & \
                              (df['DateTime'].dt.strftime('%Y-%m-%d') <= end_date) , 10, 24)
    df['Price 0%'] = np.where((df['Hinta'] < 0) , df['Hinta'], df['Hinta'] / (1 + df['alv'] / 100))


In [ ]:
def additional_columns(df: pd.DataFrame):
    calculate_alv_0_based_on_date_and_time(df)
    df['Price 24%'] = np.where((df['Hinta'] < 0) , df['Price 0%'], df['Price 0%'] * 1.24)
    df['Price 10%'] = np.where((df['Hinta'] < 0) , df['Price 0%'], df['Price 0%'] * 1.10)


In [ ]:
def create_date_subset_records(df: pd.DataFrame) -> list:
    date_subset_records: list = list()
    date_subset_records.append({'start_hour': '00', 'end_hour': '24', 'subset_name': 'Whole day'})
    date_subset_records.append({'start_hour': '00', 'end_hour': '08', 'subset_name': 'Night time'})
    date_subset_records.append({'start_hour': '20', 'end_hour': '08', 'subset_name': 'Night time'})
    date_subset_records.append({'start_hour': '08', 'end_hour': '24', 'subset_name': 'Day time'})
    i: int = -1
    for date_subset_record in date_subset_records:
        start_hour = date_subset_record['start_hour']
        end_hour = date_subset_record['end_hour']
        i += 1
        if start_hour < end_hour:
            df_subset: pd.DataFrame = df.loc[(df['DateTime'].dt.strftime('%H') < date_subset_record['end_hour']) & \
                                            (df['DateTime'].dt.strftime('%H') >= date_subset_record['start_hour'])]
        else:
            df_subset1: pd.DataFrame = df.loc[(df['DateTime'].dt.strftime('%H') < "24") & \
                                            (df['DateTime'].dt.strftime('%H') >= date_subset_record['start_hour'])]
            df_subset2: pd.DataFrame = df.loc[(df['DateTime'].dt.strftime('%H') < date_subset_record['end_hour']) & \
                                            (df['DateTime'].dt.strftime('%H') >= "0")]
            df_subset: pd.DataFrame = pd.concat([df_subset1, df_subset2])
        date_subset_record['df_subset'] = df_subset
    return date_subset_records


In [ ]:
def print_histogram(date_subset_records: list, min_max_values: tuple) -> None:
    xlim_min = min_max_values['min'] + 10
    xlim_max = min_max_values['max'] + 10
    result = plt.subplots(len(date_subset_records), figsize=(15, 12))
    fig: Figure = result[0]
    ax: np.ndarray = result[1]
    fig.tight_layout(pad=5.0)
    i: int = -1
    for date_subset_record in date_subset_records:
        i += 1
        ax[i].set_title('{} price ({} -- {}) histogram'.format(date_subset_record['subset_name'], date_subset_record['start_hour'], date_subset_record['end_hour']))
        ax[i].set_xlabel('Price (ALV included)')
        ax[i].set_ylabel('Num. of samples')
        ax[i].set_xlim(xlim_min, xlim_max)
        df_subset: pd.DataFrame = date_subset_record['df_subset']
        ax[i].hist(df_subset['Hinta'], bins=130)


In [ ]:
def print_min_average_max_table(date_subset_records: list) -> tuple:
    min_max_values: dict = {'min': None, 'max': None}
    print('{:25}{:>15}{:>15}{:>15}'.format('Subset of a day', 'Min (c€)', 'Average (c€)', 'Max (c€)'))
    i: int = -1
    for date_subset_record in date_subset_records:
        i += 1
        df_subset: pd.DataFrame = date_subset_record['df_subset']
        min_spot_price: float = df_subset['Hinta'].describe()[['min', 'mean', 'max']]
        subset_name: str = '{} ({} -- {})'.format(date_subset_record['subset_name'], \
                                                  date_subset_record['start_hour'], date_subset_record['end_hour'])
        print('{:<25}{:>15.2f}{:>15.2f}{:>15.2f}'.format(subset_name, min_spot_price['min'], \
                                                         min_spot_price['mean'], min_spot_price['max']))
        if min_max_values['min'] == None or min_max_values['min'] < min_spot_price['min']:
            min_max_values['min'] = min_spot_price['min']
        if min_max_values['max'] == None or min_max_values['max'] < min_spot_price['max']:
            min_max_values['max'] = min_spot_price['max']
    return min_max_values


In [ ]:
def print_min_average_max_table_per_month(date_subset_records: list) -> None:
    price_values:dict = dict()
    fig: Figure = plt.figure()
    print('{:25}{:15}{:>15}{:>15}{:>15}'.format('Subset of a day', 'Month', 'Min (c€)', 'Average (c€)', 'Max (c€)'))
    i: int = -1
    top: int = 0
    for date_subset_record in date_subset_records:
        i += 1
        subset_name: str = '{} ({} -- {})'.format(date_subset_record['subset_name'], \
                                                  date_subset_record['start_hour'], date_subset_record['end_hour'])
        df_subset: pd.DataFrame = date_subset_record['df_subset']
        df_subset['YearMonth'] = df_subset['DateTime'].dt.strftime('%Y-%m')
        df_subset_group_by: pd.DataFrame = df_subset.groupby('YearMonth')
        for group in df_subset_group_by:
            min_spot_price: float = group[1]['Hinta'].describe()[['min', 'mean', 'max']]
            if subset_name not in price_values:
                price_values[subset_name]: dict = dict()
            price_values[subset_name][group[0]]: float = min_spot_price['mean']
            if min_spot_price['mean'] > top:
                top: int = int(min_spot_price['mean'])
            print('{:<25}{:<15}{:>15.2f}{:>15.2f}{:>15.2f}'.format(subset_name, group[0], min_spot_price['min'], \
                                                             min_spot_price['mean'], min_spot_price['max']))
        print('\n')
    plt.title('Average spot price')
    plt.xlabel('Month')
    plt.ylabel('Price (c€)')
    plt.xticks(rotation=45)
    plt.ylim(bottom=0, top=top + 10)
    for subset_name in price_values:
        lists: dict_items = price_values[subset_name].items()
        x, y = zip(*lists)
        plt.plot(x, y, label=subset_name)
    plt.legend()
    plt.show()


In [ ]:
def print_header(df: pd.DataFrame) -> None:
    start_time: str = df['DateTime'].iloc[0]
    end_time: str = df['DateTime'].iloc[-1]
    print('Start time: {}'.format(start_time))
    print('End time:   {}'.format(end_time))
    print('\n\n')


In [ ]:
def print_spot_price_details(df: pd.DataFrame):
    print_header(df)
    date_subset_records = create_date_subset_records(df)
    print('\n\n')
    min_max_values = print_min_average_max_table(date_subset_records)
    print('\n\n')
    print_min_average_max_table_per_month(date_subset_records)
    print('\n\n')
    print_histogram(date_subset_records, min_max_values)


In [ ]:
def save_preprocessed_spot_price_df(df: pd.DataFrame):
    file_path: str = os.path.join(preprocessed_folder, spot_price_file_name_gzip)
    df.to_parquet(file_path, compression='gzip')
    file_path: str = os.path.join(preprocessed_folder, spot_price_file_name_csv)
    df.to_csv(file_path)


In [ ]:
def create_spot_price_subset(df: pd.DataFrame, subset_dates: dict) -> pd.DataFrame:
    df_spot_price_subset: pd.DataFrame = \
        df.loc[(df['DateTime'].dt.strftime('%Y-%m-%d') >= subset_dates['start_date']) & \
               (df['DateTime'].dt.strftime('%Y-%m-%d') <= subset_dates['end_date'])]
    subset_dates['start_date']: str = df_spot_price_subset['DateTime'].iloc[0].strftime('%Y-%m')
    subset_dates['end_date']: str = df_spot_price_subset['DateTime'].iloc[-1].strftime('%Y-%m')
    return df_spot_price_subset


In [ ]:
df_spot_price: pd.DataFrame = create_spot_price_dataframe()
convert_column_types(df_spot_price)
additional_columns(df_spot_price)
save_preprocessed_spot_price_df(df_spot_price)
df_spot_price_subset: pd.DataFrame = create_spot_price_subset(df_spot_price, subset_dates)
print('Spot price lines: {:d}'.format(df_spot_price.shape[0]))
print('Spot price lines: {:d}'.format(df_spot_price_subset.shape[0]))


In [ ]:
df_spot_price.dtypes

In [ ]:
# print_spot_price_details(df_spot_price)
print_spot_price_details(df_spot_price_subset)


In [ ]:
print_header(df_spot_price_subset)
date_subset_records = create_date_subset_records(df_spot_price_subset)
print('\n\n')
min_max_values = print_min_average_max_table(date_subset_records)
print('\n\n')
print('\n\n')


In [ ]:
print_header(df_spot_price_subset)
print_min_average_max_table_per_month(date_subset_records)


In [ ]:
print_header(df_spot_price_subset)
print_histogram(date_subset_records, min_max_values)


In [ ]:
df_spot_price_subset[1820:1840]


In [ ]:
df_spot_price_subset[4400:4450]


In [ ]:
df_spot_price[df_spot_price['Hinta'] <= -3.00]


In [ ]:
df_spot_price[df_spot_price['Hinta'] <= -3.00]


In [ ]:
df_spot_price.tail()